In [14]:
! pip install sqlalchemy tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [15]:
from sqlalchemy import create_engine, text
from tabulate import tabulate

### Database credentials

In [ ]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

### Construct the database URL

In [2]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [5]:
engine = create_engine(DATABASE_URL)

### Test the connection

In [6]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar())
except Exception as e:
    print("Connection failed:", e)
    raise 

Connection successful: 1


# --- CREATE DATABASE OBJECTS ---

## 1. Create Table

In [22]:
try:
    with engine.connect() as connection:
        connection.execute(text("DROP TABLE IF EXISTS Customers;"))
         
        connection.execute(text("""
            CREATE TABLE Customers (
                CustomerID SERIAL PRIMARY KEY,
                FirstName VARCHAR(255),
                LastName VARCHAR(255),
                Email VARCHAR(255) UNIQUE
            );
        """))
        
        connection.commit()
        print("Table 'Customers' created successfully.")
except Exception as e:
    print("Error creating table:", e)

Table 'Customers' created successfully.


### Check 

In [23]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("""
            INSERT INTO Customers (FirstName, LastName, Email) VALUES
            ('John', 'Doe', 'john.doe@example.com'),
            ('Jane', 'Smith', 'jane.smith@example.com');
        """))
        
        connection.commit()
        
        print("Data inserted into Customers table.")
except Exception as e:
    print("Error inserting data:", e)

Data inserted into Customers table.


In [24]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM Customers"))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+--------------+-------------+------------+------------------------+
|   customerid | firstname   | lastname   | email                  |
+==============+=============+============+========================+
|            1 | John        | Doe        | john.doe@example.com   |
+--------------+-------------+------------+------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com |
+--------------+-------------+------------+------------------------+


## 2. Create Index

### Database Indexing

**What is it?**

An index is a special data structure that databases use to speed up data retrieval.

**Purpose:**

*   **Speed up queries:** Indexes drastically improve the performance of `SELECT` queries, especially those with `WHERE` clauses that filter data.
*   **Faster sorting & grouping:** Indexes can speed up `ORDER BY` and `GROUP BY` operations.


**Trade-offs:**

*   **Increased storage space:** Indexes consume disk space.
*   **Slower write operations:** `INSERT`, `UPDATE`, and `DELETE` operations can be slower because the index must be updated along with the table data.

**In short: Indexes make reads faster but writes slower, using more storage space. Use them wisely!**

In [36]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP INDEX IF EXISTS idx_Email;
        """))
        
        connection.execute(text("""
            CREATE INDEX idx_Email ON Customers (Email);
        """))
        
        connection.commit()
        print("Index 'idx_Email' created successfully.")
        
except Exception as e:
    print("Error creating index:", e)

Index 'idx_Email' created successfully.


## 3. Create View

### Database Views

**What is it?**

A view is a virtual table based on the result-set of an SQL query. It doesn't store data physically; instead, it stores the *query* that defines how the data is derived from one or more underlying tables.

**Purpose:**

*   **Simplified queries:** Views hide the complexity of underlying table structures and complex joins, presenting a simplified and focused data representation to users.
*   **Data security:** Views can restrict access to certain columns or rows of a table, providing a security layer by exposing only the necessary data to specific users or applications.
*   **Data consistency:** Views ensure that all users see the same data based on the defined query, maintaining consistency across different applications.
*   **Data abstraction:**  Views decouple applications from the underlying table structure. If the table structure changes, you can often update the view's query without affecting the applications that use the view.
*   **Improved readability:** Complex queries can be encapsulated in views, making the code more readable and maintainable.

**Key Characteristics:**

*   **Virtual:** Views don't store data physically.
*   **Dynamic:** Data in a view is always up-to-date, reflecting the current data in the underlying tables.
*   **Read-Only (typically):**  While some databases allow updating views under certain conditions, views are generally considered read-only. You usually update the underlying tables directly, and the view reflects those changes.

**In short: Views are pre-defined queries that simplify data access, provide security, and improve code readability.**

In [35]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP VIEW IF EXISTS CustomerView;
        """))
         
        connection.execute(text("""
            CREATE VIEW CustomerView AS
            SELECT CustomerID, FirstName, LastName
            FROM Customers;
        """))
        connection.commit()
        print("View 'CustomerView' created successfully.")
except Exception as e:
    print("Error creating view:", e)

View 'CustomerView' created successfully.


In [43]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text("SELECT * FROM CustomerView"))
        
        rows = result.fetchall()
        
        column_names = result.keys()
        
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        
        print(table)

except Exception as e:
    print("Error querying CustomerView:", e)

+--------------+-------------+------------+
|   customerid | firstname   | lastname   |
+==============+=============+============+
|            1 | John        | Doe        |
+--------------+-------------+------------+
|            2 | Jane        | Smith      |
+--------------+-------------+------------+


## 4. Create Stored Procedure

## Stored Procedures (PostgreSQL: Functions)

**What is it?**

A pre-compiled set of SQL statements that are stored in the database and can be executed as a single unit. In PostgreSQL, these are called *functions*.

**Purpose:**

*   **Code reusability:** Execute the same set of SQL statements multiple times without rewriting them.
*   **Improved performance:** Stored procedures are pre-compiled, which can improve performance compared to executing the same SQL statements individually.
*   **Data integrity:** Enforce business rules and data validation within the procedure.
*   **Security:** Grant permissions to execute the procedure without granting direct access to the underlying tables.
*   **Abstraction:** Hide complex database logic from applications.

**Key Characteristics:**

*   Stored in the database.
*   Pre-compiled (generally).
*   Can accept input parameters and return output values.
*   Executed using a special command (e.g., `CALL` or `SELECT function_name()`).
*   Written in a procedural language (like PL/pgSQL in PostgreSQL).

**In short: Stored procedures are reusable, pre-compiled blocks of SQL code that enhance performance, security, and code organization.**

### --- CREATE FUNCTION ---

In [44]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("""
            DROP FUNCTION IF EXISTS GetCustomerByID(INT);
        """))

        connection.execute(text("""
            CREATE OR REPLACE FUNCTION GetCustomerByID (p_custID INT)
            RETURNS TABLE (CustomerID INT, FirstName VARCHAR(255), LastName VARCHAR(255), Email VARCHAR(255)) AS $$
            BEGIN
                RETURN QUERY SELECT * FROM Customers WHERE Customers.CustomerID = p_custID;
            END;
            $$ LANGUAGE plpgsql;
        """))
        
        connection.commit()
        
        print("Function 'GetCustomerByID' created successfully.")
        
except Exception as e:
    print("Error creating function:", e)

Function 'GetCustomerByID' created successfully.


In [45]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("""
            SELECT * FROM GetCustomerByID(:custID)
        """), {"custID": 1})

        rows = result.fetchall()
        column_names = result.keys()
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        print(table)

except Exception as e:
    print("Error using function:", e)

+--------------+-------------+------------+----------------------+
|   customerid | firstname   | lastname   | email                |
+==============+=============+============+======================+
|            1 | John        | Doe        | john.doe@example.com |
+--------------+-------------+------------+----------------------+


## 5. Create Schema

### Database Schemas

**What is it?**

A schema is a named collection of database objects (tables, views, functions, etc.).  Think of it as a namespace or a folder for organizing related objects within a single database.

**Purpose:**

*   **Organization:** Group related tables and other objects together, making the database easier to manage.
*   **Security:** Control access to groups of objects by granting permissions at the schema level.
*   **Namespace Management:**  Avoid naming conflicts when different applications or users need to use the same table names.
*   **Multi-tenancy (sometimes):**  In some cases, schemas can be used to separate data for different tenants in a multi-tenant application.

**Key Characteristics:**

*   A logical grouping of database objects.
*   Has a name.
*   Permissions can be granted at the schema level.
*   Objects within a schema are accessed using the schema name (e.g., `sales_schema.Customers`).
*   PostgreSQL has a default schema called `public`.

**In short: Schemas provide a way to organize database objects, enhance security, and avoid naming conflicts.**

In [46]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP SCHEMA IF EXISTS sales_schema CASCADE;
        """))
         
        connection.execute(text("""
            CREATE SCHEMA sales_schema;
        """))
        
        connection.commit()
        
        print("Schema 'sales_schema' created successfully.")
except Exception as e:
    print("Error creating schema:", e)

Schema 'sales_schema' created successfully.


In [47]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("""
            CREATE TABLE sales_schema.Orders (
                OrderID SERIAL PRIMARY KEY,
                CustomerID INT,
                OrderDate DATE
            );
        """))
        
        connection.commit()
        print("Table 'Orders' created in 'sales_schema'.")

        # Insert data into the table
        connection.execute(text("""
            INSERT INTO sales_schema.Orders (CustomerID, OrderDate) VALUES
            (1, '2023-01-01'),
            (2, '2023-01-05');
        """))
        connection.commit()
        print("Data inserted into 'sales_schema.Orders'.")

        # Query the table
        result = connection.execute(text("SELECT * FROM sales_schema.Orders"))
        rows = result.fetchall()
        column_names = result.keys()
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        print(table)

except Exception as e:
    print("Error using schema (creating/querying table):", e)

Table 'Orders' created in 'sales_schema'.
Data inserted into 'sales_schema.Orders'.
+-----------+--------------+-------------+
|   orderid |   customerid | orderdate   |
+===========+==============+=============+
|         1 |            1 | 2023-01-01  |
+-----------+--------------+-------------+
|         2 |            2 | 2023-01-05  |
+-----------+--------------+-------------+


## 6. Create User

## Database Users

**What is it?**

A database user represents an entity (person, application, service) that can connect to and interact with the database. Users are authenticated (usually with a username and password) to verify their identity and authorized to access specific database objects (tables, views, schemas, etc.) based on granted permissions.

**Purpose:**

*   **Authentication:** Verify the identity of the entity connecting to the database.
*   **Authorization:** Control which database objects a user can access and what actions they can perform (e.g., `SELECT`, `INSERT`, `UPDATE`, `DELETE`, `CREATE`).
*   **Security:** Limit access to sensitive data and prevent unauthorized modifications.
*   **Auditing:** Track user activity in the database for auditing and compliance purposes.

**Key Characteristics:**

*   Has a username and (usually) a password.
*   Can be granted specific privileges.
*   Can be a member of one or more roles (groups).
*   Connect to the database using a specific connection string.

**Common SQL Commands (Generally):**

*   **Create User:**
    ```sql
    CREATE USER username WITH PASSWORD 'password';
    ```
*   **Drop User:**
    ```sql
    DROP USER username;
    ```
*   **Grant Privileges:**
    ```sql
    GRANT SELECT, INSERT ON table_name TO username;
    ```
*   **Revoke Privileges:**
    ```sql
    REVOKE SELECT, INSERT ON table_name FROM username;
    ```
*   **Alter User (Change Password):**
    ```sql
    ALTER USER username WITH PASSWORD 'new_password';
    ```
*   **Connect to Database (as a User):** (This is outside of SQL - in your application or connection string)
    *   You'd specify the username and password in your database connection string (e.g., in SQLAlchemy, JDBC, etc.).

**In short: Database users are essential for security, controlling access, and auditing activity within the database.**

### --- CREATE USER ---

In [48]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP USER IF EXISTS testuser;
        """))
        
        connection.execute(text("""
            CREATE USER testuser WITH PASSWORD 'password';
        """))
        
        connection.commit()
        
        print("User 'testuser' created successfully.")
        
except Exception as e:
    print("Error creating user:", e)

User 'testuser' created successfully.


### --- GRANT PRIVILEGES ---

In [49]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("""
            GRANT SELECT ON ALL TABLES IN SCHEMA public TO testuser;
        """))
        
        connection.execute(text("""
            GRANT USAGE ON SCHEMA public TO testuser;
        """)) 
        
        connection.commit()
        
        print("Privileges granted to 'testuser'.")
        
except Exception as e:
    print("Error granting privileges:", e)

Privileges granted to 'testuser'.


In [50]:
NEW_DB_USER = "testuser"
NEW_DB_PASS = "password"

In [51]:
NEW_DATABASE_URL = f"postgresql://{NEW_DB_USER}:{NEW_DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
new_engine = create_engine(NEW_DATABASE_URL)

In [52]:
try:
    with new_engine.connect() as new_connection:
        new_result = new_connection.execute(text("SELECT * FROM Customers"))

        rows = new_result.fetchall()
        column_names = new_result.keys()
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        print("Access Customers as 'testuser':\n", table)


except Exception as e:
    print("Error accessing Customers table as 'testuser':", e)

Access Customers as 'testuser':
 +--------------+-------------+------------+------------------------+
|   customerid | firstname   | lastname   | email                  |
+==============+=============+============+========================+
|            1 | John        | Doe        | john.doe@example.com   |
+--------------+-------------+------------+------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com |
+--------------+-------------+------------+------------------------+


### Checking Access Permissions: Sales Schema - Test User

In [53]:
try:
    with new_engine.connect() as new_connection:

        new_result = new_connection.execute(text("SELECT * FROM sales_schema.Orders"))
        
        rows = new_result.fetchall()
        
        column_names = new_result.keys()
        
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        
        print("Success on Sales Schema")
        print("Test user could successfully access Sales schema\n", table)
        for row in new_result:
            print(row)

except Exception as e:
    print("User could NOT access table in 'sales_schema' (as expected):", e)

User could NOT access table in 'sales_schema' (as expected): (psycopg2.errors.InsufficientPrivilege) permission denied for schema sales_schema
LINE 1: SELECT * FROM sales_schema.Orders
                      ^

[SQL: SELECT * FROM sales_schema.Orders]
(Background on this error at: https://sqlalche.me/e/20/f405)
